In [183]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torchvision import datasets, models, transforms #画像用データセット諸々

import numpy as np
import argparse
import json
from logging.config import dictConfig
from logging import getLogger
import os
import time

In [146]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # [3, 32, 32] => [6, 28, 28]
        self.conv2 = nn.Conv2d(6, 16, 5) # [6, 28, 28] => [16, 24, 24]
        self.pool = nn.MaxPool2d(2, 2) # [N,C,H,W] => [N, C, H/2, W/2]
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [180]:
def train(model, device, train_loader, optimizer, criterion, logger):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            logger.debug("[train] batch : %s/%s (%.0f%%),\tloss : %.6f",
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())
    return (None, loss.item())

In [187]:
def test(model, device, test_loader, criterion, logger):
    model.eval()
    test_loss = []
    correct = 0
    result = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss.append(criterion(output, target).item())  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            result += torch.eq(torch.max(output, 1).indices, target)
            
    logger.debug("[dump] %s", test_loss)
    test_loss = np.mean(np.array(test_loss))
    accuracy = 100. * correct / len(test_loader.dataset)
    
    logger.debug("[test] ave loss : %.4f,\taccu : %d/%d(%.0f%%)",
        test_loss, correct, len(test_loader.dataset), accuracy)
    
    return (torch.tensor(result).numpy(), (test_loss, accuracy))

In [188]:
def main():
    with open('logging.json') as f:
        dictConfig(json.load(f))
    logger = getLogger('env')
    logger.debug("<" * 40)
    logger.info("[system] start")
    
    start_time = time.time()

    def fetch_args(args=[]):
        parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
        parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                            help='input batch size for training (default: 64)')
        parser.add_argument('--epochs', type=int, default=14, metavar='N',
                            help='number of epochs to train (default: 14)')
        parser.add_argument('--save-model', action='store_true', default=False,
                            help='For Saving the current Model')
        return parser.parse_args(args=args)
  
    args = fetch_args(["--epochs", "1", "--batch-size", "16"])
    for arg in vars(args):
        logger.info("[argparse] %s=%s", arg, vars(args)[arg])
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    logger.info("[device] %s", device)
    
    #画像の変形処理
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    #CIFAR-10のtrain, testsetのロード
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    
    model_X = CNN().to(device)
    model_Y = CNN().to(device)
    optimizer_X = optim.SGD(model_X.parameters(), lr=0.001, momentum=0.9)
    optimizer_Y = optim.SGD(model_Y.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    
    # init
    N = int(len(trainset) / 2)
    indices = np.arange(2 * N)
    A, B = indices[:N], indices[N:]
    
    log = {"X_train":[], "X_test":[], "Y_train":[], "Y_test":[], "X_acc":[], "Y_acc":[]}

    for epoch in range(args.epochs):
        logger.debug("-" * 20)
        logger.info("[system] epoch %d", epoch)
        
        # Split
        A_loader = torch.utils.data.DataLoader(
            torch.utils.data.Subset(trainset, A), batch_size=args.batch_size)
        B_loader = torch.utils.data.DataLoader(
            torch.utils.data.Subset(trainset, B), batch_size=args.batch_size)
        
        # test B
        logger.info("[system] train_a -> test_b")
        (_, loss_XA) = train(model_X, device, A_loader, optimizer_X, criterion, logger)
        (testB, (loss_XB, acc_X)) = test(model_X, device, B_loader, criterion, logger)
        
        # test A
        logger.info("[system] train_b -> test_a")
        (_, loss_YB) = train(model_Y, device, B_loader, optimizer_Y, criterion, logger)
        (testA, (loss_YA, acc_Y)) = test(model_Y, device, A_loader, criterion, logger)
        
        # Swap
        A_o, A_x = A[testA], A[np.logical_not(testA)]
        B_o, B_x = B[testB], B[np.logical_not(testB)]
        s = min(len(A_x), len(B_o))
        A = np.concatenate([A_o, B_o[:s], A_x[s:]])
        B = np.concatenate([B_x, A_x[:s], B_o[s:]])
        
        # log
        logger.debug("[dump] test_b : %s...", testB[:10])
        logger.debug("[dump] test_a : %s...", testA[:10])
        logger.debug("[dump] swap : B -> A : %s...", B_o[:s][:10])
        logger.debug("[dump] swap : A -> B : %s...", A_x[:s][:10])
        logger.debug("[dump] X loss : %s / %s,\tacc : %s", loss_XA, loss_XB, acc_X)
        logger.debug("[dump] Y loss : %s / %s,\tacc : %s", loss_YB, loss_YA, acc_Y)
        log["X_train"].append(loss_XA)
        log["X_test"].append(loss_XB)
        log["Y_train"].append(loss_YB)
        log["Y_test"].append(loss_YA)
        log["X_acc"].append(acc_X)
        log["Y_acc"].append(acc_Y)
    
    logger.debug("%s statistics %s", "-" * 10, "-" * 10)
    logger.debug("[stat] X train loss : %s", log["X_train"])
    logger.debug("[stat] X test loss : %s", log["X_test"])
    logger.debug("[stat] Y train loss : %s", log["Y_train"])
    logger.debug("[stat] Y test loss : %s", log["Y_test"])
    logger.debug("[stat] X accuracy : %s", log["X_acc"])
    logger.debug("[stat] Y accuracy : %s", log["Y_acc"])
    logger.debug("[stat] elapsed time : %s[s]", time.time() - start_time)
    
    if args.save_model:
        logger.info("[system] saving...")
        torch.save(model.state_dict(), "swap.pt")
    
    logger.info("[system] finish")

if __name__ == '__main__':
    main()

[system] start
[argparse] batch_size=16
[argparse] epochs=1
[argparse] save_model=False
[device] cpu


Files already downloaded and verified


[system] epoch 0
[system] train_a -> test_b
[system] train_b -> test_a
[system] finish


In [ ]:
#画像の変形処理
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#CIFAR-10のtrain, testsetのロード
#変形はtransformを適用
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

#DataLoaderの適用->これによりバッチの割り当て・シャッフルをまとめて行うことができる
#batch_sizeでバッチサイズを指定
#num_workersでいくつのコアでデータをロードするか指定(デフォルトはメインのみ)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=False, num_workers=2)

In [79]:
r = [torch.tensor(False), torch.tensor(True)]
torch.tensor(r).numpy()


array([False,  True])

In [85]:
import csv

log = {"A_acc":["a", "bb"], "B_acc":[1, 2]}

with open("stock.csv", "w", encoding="utf-8") as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerows(log)

In [102]:
from logging import getLogger, StreamHandler, DEBUG
logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(DEBUG)
logger.setLevel(DEBUG)
logger.addHandler(handler)
logger.propagate = False

print("a")

a


In [121]:
# from logging import getLogger
logger = getLogger(__name__)

from logging import StreamHandler, basicConfig, DEBUG, getLogger, Formatter

def setup_logger(log_filename):
    format_str = '%(asctime)s@%(name)s %(levelname)s # %(message)s'
    basicConfig(filename=log_filename, level=DEBUG, format=format_str)
    stream_handler = StreamHandler()
    stream_handler.setFormatter(Formatter(format_str))
    getLogger().addHandler(stream_handler)
    

logger.info("messagea")

logger.info("mess")


messagea
messagea
messagea
messagea
messagea
messagea
messagea
messagea
messagea
messagea
mess
mess
mess
mess
mess
mess
mess
mess
mess
mess


In [120]:
setup_logger("test")

In [124]:
from logging import StreamHandler, basicConfig, INFO, getLogger, Formatter, shutdown

def setup_logger(log_filename):
    format_str = '%(asctime)s@%(name)s %(levelname)s # %(message)s'
    basicConfig(filename=log_filename, level=INFO, format=format_str)
    stream_handler = StreamHandler()
    stream_handler.setFormatter(Formatter(format_str))
    getLogger().addHandler(stream_handler)

def kill_logger():
    # for l in getLogger(__name__):
    # logger = loggers.get(l)
    for h in logger.handlers:
        logger.removeHandler(h)
    # getLogger().removeHandler()
    shutdown()
    return

logger = getLogger(__name__)
setup_logger("log.txt")

logger.info("aa")
print(logger)
# kill_logger()

aa
aa
aa
aa
aa
aa
aa
aa
aa
aa


<Logger __main__ (DEBUG)>


In [144]:
import json
from logging.config import dictConfig

with open('logging.json') as f:
    dictConfig(json.load(f))


# 動作確認
from logging import getLogger

# RootLogger: error のみが標準エラーに
root_logger = getLogger()
root_logger.debug('RootLogger: debug')
root_logger.error('RootLogger: error')

# myapp: debug は指定したファイルに
# myapp: error は指定したファイルと標準エラーに
myapp_logger = getLogger('env')
myapp_logger.debug('myapp: debug')
myapp_logger.error('myapp: error')
myapp_logger = getLogger('env.model')
myapp_logger.error('env: error')

# これはすべて無視される
hoge_logger = getLogger('hoge.fuga.piyo')
hoge_logger.debug('hoge: debug')
hoge_logger.error('hoge: error')

2020-04-29 19:33:53,564: RootLogger: error
2020-04-29 19:33:53,566: myapp: error
2020-04-29 19:33:53,568: env: error


In [160]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
args = parser.parse_args(['--foo', 'BAR'])
for a in vars(args):
    print(a, vars(args)[a])
print(vars(args))


foo BAR
{'foo': 'BAR'}
